In [1]:
from binance.client import Client #pip install python-binance
import pandas as pd
from datetime import datetime
import numpy as np
import random
from collections import deque
from sklearn import preprocessing
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [2]:
seq_len = 60  
future_len = 6
prediction_coin = 'BTC-USD'

In [3]:
#enter start time as string in format 'dd Mon, yyyy'. 2 months ago works well
#enter api keys from binance as string
start_time = '15 Dec, 2019'
end_time = '15 Feb, 2020'
start_time_test = '16 Feb, 2020'
end_time_test = '28 Feb, 2020'
api_key='3VmtkFSxpzJfwsV3OvYWxgFPxblkEKwmtF20REacLOddjKIk9H1EjCEQ8b22MBZO'
api_secret='tHGaAXicd1drbxw5UjfdljqDugGcDz9EpOvGphXsiWiTSY6Mmp1W98hbxFOCRlv0s'

In [4]:
client = Client(api_key=api_key,api_secret=api_secret)

In [5]:
#this cell takes a few minutes, gathers ~90,000 prices
BTC = client.get_historical_klines(symbol='BTCUSDT',interval=Client.KLINE_INTERVAL_1MINUTE,start_str=start_time,end_str=end_time)
LTC = client.get_historical_klines(symbol='LTCUSDT',interval=Client.KLINE_INTERVAL_1MINUTE,start_str=start_time,end_str=end_time)
ETH = client.get_historical_klines(symbol='ETHUSDT',interval=Client.KLINE_INTERVAL_1MINUTE,start_str=start_time,end_str=end_time)

BTC_test = client.get_historical_klines(symbol='BTCUSDT',interval=Client.KLINE_INTERVAL_1MINUTE,start_str=start_time_test,end_str=end_time_test)
LTC_test = client.get_historical_klines(symbol='LTCUSDT',interval=Client.KLINE_INTERVAL_1MINUTE,start_str=start_time_test,end_str=end_time_test)
ETH_test = client.get_historical_klines(symbol='ETHUSDT',interval=Client.KLINE_INTERVAL_1MINUTE,start_str=start_time_test,end_str=end_time_test)

In [6]:
BTC_cols = ['time','Open','High','Low','BTC-USD_close','BTC-USD_volume','CloseTime','QuoteAssetVolume','NumberOfTrades','TBBAV','TBQAV','null']
LTC_cols = ['time','Open','High','Low','LTC-USD_close','LTC-USD_volume','CloseTime','QuoteAssetVolume','NumberOfTrades','TBBAV','TBQAV','null']
ETH_cols = ['time','Open','High','Low','ETH-USD_close','ETH-USD_volume','CloseTime','QuoteAssetVolume','NumberOfTrades','TBBAV','TBQAV','null']

In [7]:
BTC = pd.DataFrame(BTC,columns=BTC_cols)
LTC = pd.DataFrame(LTC,columns=LTC_cols)
ETH = pd.DataFrame(ETH,columns=ETH_cols)

BTC_test = pd.DataFrame(BTC_test,columns=BTC_cols)
LTC_test = pd.DataFrame(LTC_test,columns=LTC_cols)
ETH_test = pd.DataFrame(ETH_test,columns=ETH_cols)

In [8]:
BTC = BTC[['time','BTC-USD_close','BTC-USD_volume']]
LTC = LTC[['time','LTC-USD_close','LTC-USD_volume']]
ETH = ETH[['time','ETH-USD_close','ETH-USD_volume']]

BTC_test = BTC_test[['time','BTC-USD_close','BTC-USD_volume']]
LTC_test = LTC_test[['time','LTC-USD_close','LTC-USD_volume']]
ETH_test = ETH_test[['time','ETH-USD_close','ETH-USD_volume']]

In [9]:
middle_df = pd.merge(BTC,LTC,how='inner',on='time')

middle_df_test = pd.merge(BTC_test,LTC_test,how='inner',on='time')

In [10]:
df_train = pd.merge(middle_df,ETH,how='inner',on='time')
df_test = pd.merge(middle_df_test,ETH_test,how='inner',on='time')

In [11]:
df_train.set_index('time',inplace=True)
df_test.set_index('time',inplace=True)

In [12]:
for col in df_train.columns:
    df_train[col] = df_train[col].astype(np.float64)
    df_test[col] = df_test[col].astype(np.float64)

In [13]:
df_train.drop('LTC-USD_volume',axis=1,inplace=True)
df_test.drop('LTC-USD_volume',axis=1,inplace=True)

In [14]:
def up_or_down(price_0, price_1):
    if float(price_1) > float(price_0):
        return 1
    else:
        return 0

In [15]:
def make_train(df):
    df = df.drop("future", 1)

    for col in df.columns: 
        if col != "target":  
            df[col] = df[col].pct_change()
            df.dropna(inplace=True) 
            df[col] = preprocessing.scale(df[col].values) 
            
    df.dropna(inplace=True) 

    time_period_instance = [] 
    interval = deque(maxlen=seq_len)

    for i in df.values:
        interval.append([n for n in i[:-1]])
        if len(interval) == seq_len:
            time_period_instance.append([np.array(interval), i[-1]])

    random.shuffle(time_period_instance)

    up = []
    down = []
    
    for item1, item2 in time_period_instance:
        if item2 == 0:
            down.append([item1, item2])
        elif item2 == 1:
            up.append([item1, item2])
            
    random.shuffle(up)
    random.shuffle(down)

    small = min(len(up), len(down))

    up = up[:small]
    down = down[:small]

    time_period_instance = up+down
    random.shuffle(time_period_instance)

    X = []
    y = []

    for item1, item2 in time_period_instance:
        X.append(item1)
        y.append(item2)

    return np.array(X), y

In [16]:
def make_test(df):
    
    df = df.drop("future", 1)
    
    for col in df.columns: 
        if col != "target":  
            df[col] = df[col].pct_change()
            df.dropna(inplace=True) 
            df[col] = preprocessing.scale(df[col].values) 
            
    df.dropna(inplace=True) 
    
    time_period_instance = [] 
    interval = deque(maxlen=seq_len)

    for i in df.values:
        interval.append([n for n in i[:-1]])
        if len(interval) == seq_len:
            time_period_instance.append([np.array(interval), i[-1]])

    X = []
    y = []

    for item1, item2 in time_period_instance:
        X.append(item1)
        y.append(item2)

    return np.array(X), y

In [17]:
df_train.fillna(method="ffill", inplace=True)  
df_train.dropna(inplace=True)

df_train['future'] = df_train[f'{prediction_coin}_close'].shift(-future_len)
df_train['target'] = list(map(up_or_down, df_train[f'{prediction_coin}_close'], df_train['future']))

df_train.dropna(inplace=True)

times = sorted(df_train.index.values)
last = sorted(df_train.index.values)[-int(0.05*len(times))]

valid = df_train[(df_train.index >= last)]
df_train = df_train[(df_train.index < last)]

x_train, y_train = make_train(df_train)
x_valid, y_valid = make_train(valid)

In [18]:
df_test.fillna(method="ffill", inplace=True)

df_test['future'] = df_test[f'{prediction_coin}_close'].shift(-future_len)
df_test['target'] = list(map(up_or_down, df_test[f'{prediction_coin}_close'], df_test['future']))

x_test, y_test = make_test(df_test)

In [19]:
x_test.shape

(16863, 60, 5)

In [20]:
x_train[0]

array([[-1.26763547e+00,  1.03980823e+00, -1.15862097e+00,
        -6.90137984e-01, -3.09117278e-01],
       [ 1.08272331e+00, -6.61306181e-01,  3.23843339e-01,
         1.98717395e-01,  5.48820489e-04],
       [-1.18496574e+00,  5.29495015e-01,  1.58870053e-01,
        -3.48489910e-01, -2.88884265e-01],
       [ 6.23914079e-02, -2.03644265e-01,  3.23637679e-01,
         1.30326512e-01, -1.26301954e-01],
       [-9.64343059e-01, -3.63058228e-01, -5.00166318e-01,
        -9.64248480e-01, -1.60899578e-01],
       [-1.48196087e+00,  1.32931773e+00, -9.94983003e-01,
        -1.10212583e+00, -9.07582031e-02],
       [ 2.70152934e-01, -6.11780584e-01, -6.66134196e-01,
         1.99142888e-01, -3.33461008e-01],
       [ 9.71131951e-01, -3.26947395e-01,  3.24393021e-01,
         1.22716094e+00, -1.16102566e-01],
       [ 6.92758569e-01, -3.31444932e-01, -5.01298388e-01,
         4.04201593e-01, -2.04895535e-01],
       [-8.83352387e-01, -3.80368432e-01,  3.24461861e-01,
        -4.17055361e-01

In [21]:
#Use this if running on CPU
#Try different number of neurons per layer to optimize, maybe change 128 to other choices
epochs = 10
batch_size = 64

model = Sequential()
model.add(LSTM(128, input_shape=(x_train.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization()) 

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [22]:
#Use this if running on GPU
#Try different number of neurons per layer to optimize, maybe change 128 to other choices
'''
model = Sequential()
model.add(CuDNNLSTM(128, input_shape=(x_train.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization()) 

model.add(CuDNNLSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))
'''

"\nmodel = Sequential()\nmodel.add(CuDNNLSTM(128, input_shape=(x_train.shape[1:]), return_sequences=True))\nmodel.add(Dropout(0.2))\nmodel.add(BatchNormalization()) \n\nmodel.add(CuDNNLSTM(128, return_sequences=True))\nmodel.add(Dropout(0.1))\nmodel.add(BatchNormalization())\n\nmodel.add(CuDNNLSTM(128))\nmodel.add(Dropout(0.2))\nmodel.add(BatchNormalization())\n\nmodel.add(Dense(32, activation='relu'))\nmodel.add(Dropout(0.2))\n\nmodel.add(Dense(2, activation='softmax'))\n"

In [23]:
optimizer = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

In [24]:
model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_valid, y_valid),
)

Train on 83340 samples, validate on 4186 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
83340/83340 [==============================] - 687s 8ms/sample - loss: 0.7064 - acc: 0.5357 - val_loss: 0.6861 - val_acc: 0.5468
Epoch 2/10
83340/83340 [==============================] - 607s 7ms/sample - loss: 0.6840 - acc: 0.5564 - val_loss: 0.6868 - val_acc: 0.5490
Epoch 3/10
83340/83340 [==============================] - 594s 7ms/sample - loss: 0.6808 - acc: 0.5624 - val_loss: 0.6890 - val_acc: 0.5590
Epoch 4/10
83340/83340 [==============================] - 585s 7ms/sample - loss: 0.6785 - acc: 0.5709 - val_loss: 0.6909 - val_acc: 0.5557
Epoch 5/10
83340/83340 [==============================] - 598s 7ms/sample - loss: 0.6722 - acc: 0.5817 - val_loss: 0.6951 - val_acc: 0.5564
Epoch 6/10
83340/83340 [==============================] - 634s 8ms/sample - loss: 0.6599 - acc: 0.6054 - val_loss: 0.7044 - val_acc: 0.5452
Epoch 7/10
83340/83340 [==============================] - 591s 

In [25]:
predictions = model.predict(x_test)

In [26]:
predictions_final = []
for i in range(0,len(predictions)):
    if predictions[i][0] < predictions[i][1]:
        predictions_final.append(1)
    else:
        predictions_final.append(0)

In [27]:
print(classification_report(y_test,predictions_final))

              precision    recall  f1-score   support

         0.0       0.55      0.58      0.56      8600
         1.0       0.53      0.49      0.51      8263

    accuracy                           0.54     16863
   macro avg       0.54      0.54      0.54     16863
weighted avg       0.54      0.54      0.54     16863



In [28]:
#add scaled columns as new columns so we can still use unscaled data
df_test = df_test.drop("future", 1)

for col in df_test.columns: 
    if col != "target":  
        df_test[f'{col}_scaled'] = df_test[col].pct_change()
        df_test.dropna(inplace=True) 
        df_test[f'{col}_change_scaled'] = preprocessing.scale(df_test[col].values) 
            
    df_test.dropna(inplace=True)

In [29]:
df_test

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,ETH-USD_close,ETH-USD_volume,target,BTC-USD_close_scaled,BTC-USD_close_change_scaled,BTC-USD_volume_scaled,BTC-USD_volume_change_scaled,LTC-USD_close_scaled,LTC-USD_close_change_scaled,ETH-USD_close_scaled,ETH-USD_close_change_scaled,ETH-USD_volume_scaled,ETH-USD_volume_change_scaled
time,,,,,,,,,,,,,,,,
1581811500000,9882.88,20.709930,76.96,263.28,363.50504,0,0.000245,0.791333,-0.865544,-0.340031,0.002083,0.920020,0.000874,0.351477,-0.263436,-0.300179
1581811560000,9880.87,26.135730,76.91,262.67,635.20889,0,-0.000203,0.785704,0.261990,-0.249523,-0.000650,0.909986,-0.002317,0.310775,0.747456,0.021934
1581811620000,9874.01,47.564872,76.67,261.45,2465.87809,1,-0.000694,0.766492,0.819917,0.107937,-0.003121,0.861819,-0.004645,0.229371,2.881996,2.192249
1581811680000,9874.12,34.626326,76.62,261.37,976.81107,1,0.000011,0.766800,-0.272019,-0.107891,-0.000652,0.851784,-0.000306,0.224033,-0.603869,0.426914
1581811740000,9870.00,38.518121,76.65,260.67,1456.45063,1,-0.000417,0.755261,0.112394,-0.042972,0.000392,0.857805,-0.002678,0.177326,0.491026,0.995542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1582847760000,8805.57,43.825564,61.63,226.91,1496.62529,0,0.000031,-2.225739,-0.225757,0.045562,0.002929,-2.156610,0.003716,-2.075303,-0.266958,1.043170
1582847820000,8807.05,47.205806,61.56,227.01,845.38744,0,0.000168,-2.221594,0.077129,0.101948,-0.001136,-2.170658,0.000441,-2.068630,-0.435138,0.271108
1582847880000,8808.45,79.121986,61.55,227.14,1455.99312,0,0.000159,-2.217673,0.676107,0.634343,-0.000162,-2.172665,0.000573,-2.059956,0.722279,0.994999


In [30]:
blank = []
for i in range(0,59):
    blank.append('NaN')
for i in range(0,len(predictions_final)):
    blank.append(predictions_final[i])

In [31]:
df_test['Predicted'] = blank

In [32]:
#these two prints should match, if not the df is lined up wrong
#remove last 3 because they were autofilled as there was no future data
print(accuracy_score(list(df_test[seq_len-1:-future_len]['Predicted']),list(df_test[seq_len-1:-future_len]['target'])))
print(accuracy_score(predictions_final[:-future_len],y_test[:-future_len]))

0.539835083348164
0.539835083348164


In [33]:
print(accuracy_score(list(df_test[seq_len-1:-future_len]['target']),y_test[:-future_len]))

1.0


In [35]:
#first 60 rows Predict NaN because NN needs 60 time series inputs to predict
#last 3 rows automatically wrong, no future data provided
df_test

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,ETH-USD_close,ETH-USD_volume,target,BTC-USD_close_scaled,BTC-USD_close_change_scaled,BTC-USD_volume_scaled,BTC-USD_volume_change_scaled,LTC-USD_close_scaled,LTC-USD_close_change_scaled,ETH-USD_close_scaled,ETH-USD_close_change_scaled,ETH-USD_volume_scaled,ETH-USD_volume_change_scaled,Predicted
time,,,,,,,,,,,,,,,,,
1581811500000,9882.88,20.709930,76.96,263.28,363.50504,0,0.000245,0.791333,-0.865544,-0.340031,0.002083,0.920020,0.000874,0.351477,-0.263436,-0.300179,NaN
1581811560000,9880.87,26.135730,76.91,262.67,635.20889,0,-0.000203,0.785704,0.261990,-0.249523,-0.000650,0.909986,-0.002317,0.310775,0.747456,0.021934,NaN
1581811620000,9874.01,47.564872,76.67,261.45,2465.87809,1,-0.000694,0.766492,0.819917,0.107937,-0.003121,0.861819,-0.004645,0.229371,2.881996,2.192249,NaN
1581811680000,9874.12,34.626326,76.62,261.37,976.81107,1,0.000011,0.766800,-0.272019,-0.107891,-0.000652,0.851784,-0.000306,0.224033,-0.603869,0.426914,NaN
1581811740000,9870.00,38.518121,76.65,260.67,1456.45063,1,-0.000417,0.755261,0.112394,-0.042972,0.000392,0.857805,-0.002678,0.177326,0.491026,0.995542,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1582847760000,8805.57,43.825564,61.63,226.91,1496.62529,0,0.000031,-2.225739,-0.225757,0.045562,0.002929,-2.156610,0.003716,-2.075303,-0.266958,1.043170,1
1582847820000,8807.05,47.205806,61.56,227.01,845.38744,0,0.000168,-2.221594,0.077129,0.101948,-0.001136,-2.170658,0.000441,-2.068630,-0.435138,0.271108,1
1582847880000,8808.45,79.121986,61.55,227.14,1455.99312,0,0.000159,-2.217673,0.676107,0.634343,-0.000162,-2.172665,0.000573,-2.059956,0.722279,0.994999,1


In [36]:
df_test['DecodedTime'] = df_test.index
for i in range(len(df_test)):
    df_test['DecodedTime'].iloc[i] = datetime.fromtimestamp(df_test['DecodedTime'].iloc[i]/1000)

C:\Users\fq1228hj\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [37]:
#Removes first 60 and last 3 entries that can't be predicted for backtesting
backtest_df = df_test.iloc[59:-future_len]

In [38]:
backtest_df

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,ETH-USD_close,ETH-USD_volume,target,BTC-USD_close_scaled,BTC-USD_close_change_scaled,BTC-USD_volume_scaled,BTC-USD_volume_change_scaled,LTC-USD_close_scaled,LTC-USD_close_change_scaled,ETH-USD_close_scaled,ETH-USD_close_change_scaled,ETH-USD_volume_scaled,ETH-USD_volume_change_scaled,Predicted,DecodedTime
time,,,,,,,,,,,,,,,,,,
1581815040000,9929.01,7.659028,78.04,265.67,478.64259,0,0.000125,0.920522,-0.675808,-0.557734,-0.001791,1.136769,-0.001841,0.510949,-0.647188,-0.163680,0,2020-02-15 19:04:00
1581815100000,9927.16,6.534654,78.04,265.93,70.29685,0,-0.000186,0.915341,-0.146804,-0.576490,0.000000,1.136769,0.000979,0.528298,-0.853133,-0.647786,0,2020-02-15 19:05:00
1581815160000,9919.23,35.095332,77.95,265.57,234.14131,1,-0.000799,0.893133,4.370649,-0.100068,-0.001153,1.118707,-0.001354,0.504277,2.330751,-0.453544,1,2020-02-15 19:06:00
1581815220000,9921.01,9.035562,77.98,265.91,112.65745,1,0.000179,0.898118,-0.742542,-0.534772,0.000385,1.124727,0.001280,0.526963,-0.518848,-0.597566,1,2020-02-15 19:07:00
1581815280000,9919.60,10.897427,77.92,265.83,61.43789,1,-0.000142,0.894169,0.206060,-0.503714,-0.000769,1.112686,-0.000301,0.521625,-0.454649,-0.658289,1,2020-02-15 19:08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1582847400000,8760.15,13.579751,61.31,224.56,878.35858,1,-0.000662,-2.352940,-0.516684,-0.458970,-0.001303,-2.220832,-0.002842,-2.232106,-0.173369,0.310196,1,2020-02-27 17:50:00
1582847460000,8761.65,35.871948,61.21,224.52,591.72211,1,0.000171,-2.348739,1.641576,-0.087113,-0.001631,-2.240901,-0.000178,-2.234775,-0.326332,-0.029620,1,2020-02-27 17:51:00
1582847520000,8769.10,24.846951,61.20,224.63,384.13678,1,0.000850,-2.327875,-0.307343,-0.271022,-0.000163,-2.242908,0.000490,-2.227435,-0.350816,-0.275719,1,2020-02-27 17:52:00


In [39]:
#set up cols of buy/sell df
cols = ['BuyPrice','SellPrice']
buy_price = []
sell_price = []
track_buy_sell = 0 #0 indicates money is in USDT, 1 indicates money is held in desired ratio
for i in range(0,len(backtest_df)):
    if backtest_df['Predicted'].iloc[i] == 1 and track_buy_sell == 0:
        #buy
        buy_price.append(backtest_df[f'{prediction_coin}_close'].iloc[i])
        sell_price.append(np.NAN)
        track_buy_sell = 1
    elif backtest_df['Predicted'].iloc[i] == 0 and track_buy_sell == 1:
        #sell
        buy_price.append(np.NAN)
        sell_price.append(backtest_df[f'{prediction_coin}_close'].iloc[i])
        track_buy_sell = 0
    else:
        #record nothing happened by appending nulls
        buy_price.append(np.NAN)
        sell_price.append(np.NAN)

In [40]:
buy_sell_df = pd.DataFrame([buy_price,sell_price])
buy_sell_df = buy_sell_df.T
buy_sell_df.columns=cols
buy_sell_df.index = backtest_df['DecodedTime']

In [41]:
#backtest over a time period where the price didn't change
#buy_sell_df = buy_sell_df['2020-02-17 06:09:00':'2020-02-24 18:37:00'] 

In [42]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
  #  print(buy_sell_df)

In [43]:
#this does not consider trading fees
money_owned_USD = 1
track_buy_sell = 0
coin_owned = 0
number_of_trades = 0
for i in range(0,len(buy_sell_df)):
    #buy
    if np.isnan(buy_sell_df['BuyPrice'].iloc[i]) == False and track_buy_sell == 0:
        #actual amount
        coin_owned = money_owned_USD/buy_sell_df['BuyPrice'].iloc[i]
        money_owned_USD = 0
        track_buy_sell = 1
        number_of_trades += 1
    #sell
    if np.isnan(buy_sell_df['SellPrice'].iloc[i]) == False and track_buy_sell == 1:
        #actual amount
        money_owned_USD = coin_owned*buy_sell_df['SellPrice'].iloc[i]
        coin_owned = 0
        track_buy_sell = 0
        number_of_trades += 1
#for sake of backtesting, if last trade was a buy, undo
i=0
while coin_owned != 0:
    if np.isnan(buy_sell_df['BuyPrice'].iloc[-i]) == False:
        money_owned_USD = coin_owned*buy_sell_df['BuyPrice'].iloc[-i]
        coin_owned = 0
    i+=1

In [44]:
#precent growth (without fees)
money_owned_USD

1.1179471173068332

In [45]:
number_of_trades

1777